## Single Tool

In [1]:
from langchain.tools import BaseTool
from typing import Optional, Type
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain.chat_models import ChatOpenAI
from pydantic import BaseModel, Field
import yfinance as yf

In [2]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

Enter your OpenAI API key:  ········


In [3]:
def get_stock_price (symbol):
    ticker = yf.Ticker(symbol)
    todays_data = ticker.history(period='id')
    return round(todays_data['Close'][0], 2)
    
# use the function
print (get_stock_price('AAPL'))

class StockPriceCheckInput(BaseModel):
    """Input for Stock price check."""

    stockticker: str = Field(..., description="Ticker symbol for stock or index")

class StockPriceTool(BaseTool):
    name = "get_stock_ticker_price"
    description = "Useful for when you need to find out the price of stock. You should input the stock ticker used on the yfinance API"

    def _run(self, stockticker: str):
        # print("i'm running")
        price_response = get_stock_price(stockticker)

        return price_response

    def _arun(self, stockticker: str):
        raise NotImplementedError("This tool does not support async")

    args_schema: Optional[Type[BaseModel]] = StockPriceCheckInput

tools = [StockPriceTool()]

224.31


/home/hunglv/miniconda3/envs/llm_py310/lib/python3.10/site-packages/yfinance/utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
/tmp/ipykernel_1277615/2413667966.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return round(todays_data['Close'][0], 2)


In [4]:
llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613", )

/home/hunglv/miniconda3/envs/llm_py310/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [5]:
open_ai_agent = initialize_agent(tools,
                        llm,
                        agent=AgentType.OPENAI_FUNCTIONS,
                        verbose=True)

/home/hunglv/miniconda3/envs/llm_py310/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.3.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [6]:
open_ai_agent.run("What is the price of Google stock?")

/home/hunglv/miniconda3/envs/llm_py310/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...

Invoking: `get_stock_ticker_price` with `{'stockticker': 'GOOGL'}`


177.66

/home/hunglv/miniconda3/envs/llm_py310/lib/python3.10/site-packages/yfinance/utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
/tmp/ipykernel_1277615/2413667966.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return round(todays_data['Close'][0], 2)


The current price of Google stock (GOOGL) is $177.66.

> Finished chain.


'The current price of Google stock (GOOGL) is $177.66.'

In [7]:
open_ai_agent.run("What is the price of Bitcoin?")



> Entering new AgentExecutor chain...

Invoking: `get_stock_ticker_price` with `{'stockticker': 'BTC-USD'}`


66481.7

/home/hunglv/miniconda3/envs/llm_py310/lib/python3.10/site-packages/yfinance/utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
/tmp/ipykernel_1277615/2413667966.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return round(todays_data['Close'][0], 2)


The current price of Bitcoin is $66,481.70.

> Finished chain.


'The current price of Bitcoin is $66,481.70.'

In [8]:
open_ai_agent.run("What is the price of ETH?")



> Entering new AgentExecutor chain...

Invoking: `get_stock_ticker_price` with `{'stockticker': 'ETH-USD'}`


3499.71

/home/hunglv/miniconda3/envs/llm_py310/lib/python3.10/site-packages/yfinance/utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
/tmp/ipykernel_1277615/2413667966.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return round(todays_data['Close'][0], 2)


The current price of ETH is $3499.71.

> Finished chain.


'The current price of ETH is $3499.71.'

## OpenAI Functions Agent

### Advantages
- Better reasoning and tool selection than ReACT / Toolformer?
- Less tokens needed

### Disadvantages
- Not as easy to customize via prompt change etc if it doesn't work
- locks you code into the OpenAI way
- Still need tokens for Tool/Function descriptions

## Multiple Tools

In [15]:
from datetime import datetime, timedelta

def get_price_change_percent(symbol, days_ago):
    ticker = yf.Ticker(symbol)

    # Get today's date
    end_date = datetime.now()

    # Get the date N days ago
    start_date = end_date - timedelta(days=days_ago)

    # Convert dates to string format that yfinance can accept
    start_date = start_date.strftime('%Y-%m-%d')
    end_date = end_date.strftime('%Y-%m-%d')

    # Get the historical data
    historical_data = ticker.history(start=start_date, end=end_date)

    # Get the closing price N days ago and today's closing price
    old_price = historical_data['Close'].iloc[0]
    new_price = historical_data['Close'].iloc[-1]

    # Calculate the percentage change
    percent_change = ((new_price - old_price) / old_price) * 100

    return round(percent_change, 2)

# use the function
get_price_change_percent("AAPL", 30) # for 30 days ago

/home/hunglv/miniconda3/envs/llm_py310/lib/python3.10/site-packages/yfinance/utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')


6.98

In [17]:
import yfinance as yf
from datetime import datetime, timedelta

def calculate_performance(symbol, days_ago):
    ticker = yf.Ticker(symbol)
    end_date = datetime.now()
    start_date = end_date - timedelta(days=days_ago)
    start_date = start_date.strftime('%Y-%m-%d')
    end_date = end_date.strftime('%Y-%m-%d')
    historical_data = ticker.history(start=start_date, end=end_date)
    old_price = historical_data['Close'].iloc[0]
    new_price = historical_data['Close'].iloc[-1]
    percent_change = ((new_price - old_price) / old_price) * 100
    return round(percent_change, 2)

def get_best_performance(stocks, days_ago):
    best_stock = None
    best_performance = None
    for stock in stocks:
        try:
            performance = calculate_performance(stock, days_ago)
            if best_performance is None or performance > best_performance:
                best_stock = stock
                best_performance = performance
        except Exception as e:
            print(f"Could not calculate performance for {stock}: {e}")
    return best_stock, best_performance

stocks = ['AAPL', 'MSFT', 'GOOG']
days_ago = 90

get_best_performance(stocks, days_ago)

/home/hunglv/miniconda3/envs/llm_py310/lib/python3.10/site-packages/yfinance/utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
/home/hunglv/miniconda3/envs/llm_py310/lib/python3.10/site-packages/yfinance/utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
/home/hunglv/miniconda3/envs/llm_py310/lib/python3.10/site-packages/yfinance/utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')


('AAPL', 35.44)

### Make the tools

In [34]:

class StockPriceCheckInput(BaseModel):
    """Input for Stock price check."""

    stockticker: str = Field(..., description="Ticker symbol for stock or index")

class StockPriceTool(BaseTool):
    name = "get_stock_ticker_price"
    description = "Useful for when you need to find out the price of stock. You should input the stock ticker used on the yfinance API"

    def _run(self, stockticker: str):
        # print("i'm running")
        price_response = get_stock_price(stockticker)

        return price_response

    def _arun(self, stockticker: str):
        raise NotImplementedError("This tool does not support async")

    args_schema: Optional[Type[BaseModel]] = StockPriceCheckInput


class StockChangePercentageCheckInput(BaseModel):
    """Input for Stock ticker check. for percentage check"""
    stockticker: str = Field(..., description="Ticker symbol for stock or index")
    days_ago: int  = Field(..., description="Int number of days to look back")

class StockChangePercentageTool(BaseTool):
    name = 'get_stock_ticker_change'
    description = "Useful for when you need to find out the percentage change in a stock's value. You should input the stock ticker used on the yfinance API and also input the number of days to check the change over"

    def _run(self, stockticker: str, days_ago: int):
        percentage = calculate_performance(stockticker, days_ago)
        return percentage

    def _arun(self, stockticker: str):
        raise NotImplementedError("This tool does not support async")

    args_schema: Optional[Type[BaseModel]] = StockChangePercentageCheckInput
    
class StockBestPerformingInput(BaseModel):
    """Input for Best performance stock check"""
    stocks: list = Field(..., description="List of stocks")
    days_ago: int  = Field(..., description="Number of day")
    
class StockGetBestPerformingTool(BaseTool):
    name = 'get_best_stock_performance'
    description = "Useful for when you need to find best performance of multiple stocks over a period. You should input the list of stocks used on the yfinance API and also input the number of days to check the change over"

    def _run(self, stocks: list, days_ago: int):
        percentage = get_best_performance(stocks, days_ago)
        return percentage

    def _arun(self, stockticker: str):
        raise NotImplementedError("This tool does not support async")

    args_schema: Optional[Type[BaseModel]] = StockBestPerformingInput


tools = [StockPriceTool(), StockChangePercentageTool(), StockGetBestPerformingTool()]

In [35]:
llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613")

open_ai_agent = initialize_agent(tools,
                        llm,
                        agent=AgentType.OPENAI_FUNCTIONS,
                        verbose=True)


In [36]:
open_ai_agent.run("What is the price of Google stock today?")



> Entering new AgentExecutor chain...

Invoking: `get_stock_ticker_price` with `{'stockticker': 'GOOGL'}`


177.66

/home/hunglv/miniconda3/envs/llm_py310/lib/python3.10/site-packages/yfinance/utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
/tmp/ipykernel_1277615/2413667966.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return round(todays_data['Close'][0], 2)


The price of Google stock today is $177.66.

> Finished chain.


'The price of Google stock today is $177.66.'

In [37]:
open_ai_agent.run("Has google's stock gone up over the past 90 days?")



> Entering new AgentExecutor chain...

Invoking: `get_stock_ticker_change` with `{'stockticker': 'GOOGL', 'days_ago': 90}`


13.81

/home/hunglv/miniconda3/envs/llm_py310/lib/python3.10/site-packages/yfinance/utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')


Yes, Google's stock (GOOGL) has gone up by 13.81% over the past 90 days.

> Finished chain.


"Yes, Google's stock (GOOGL) has gone up by 13.81% over the past 90 days."

In [32]:
open_ai_agent.run("How much has google's stock gone up over the past 3 months?")



> Entering new AgentExecutor chain...

Invoking: `get_stock_ticker_change` with `{'stockticker': 'GOOGL', 'days_ago': 90}`


13.81

/home/hunglv/miniconda3/envs/llm_py310/lib/python3.10/site-packages/yfinance/utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')


Google's stock has gone up by 13.81% over the past 3 months.

> Finished chain.


"Google's stock has gone up by 13.81% over the past 3 months."

In [38]:
open_ai_agent.run("Which stock out of Google, Meta and MSFT has performed best over the past 3 months?")



> Entering new AgentExecutor chain...

Invoking: `get_best_stock_performance` with `{'stocks': ['GOOGL', 'META', 'MSFT'], 'days_ago': 90}`




/home/hunglv/miniconda3/envs/llm_py310/lib/python3.10/site-packages/yfinance/utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
/home/hunglv/miniconda3/envs/llm_py310/lib/python3.10/site-packages/yfinance/utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
/home/hunglv/miniconda3/envs/llm_py310/lib/python3.10/site-packages/yfinance/utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')


('GOOGL', 13.81)Google (GOOGL) has performed the best over the past 3 months with a return of 13.81%.

> Finished chain.


'Google (GOOGL) has performed the best over the past 3 months with a return of 13.81%.'

In [39]:
open_ai_agent.run("How much has MSFT's stock gone up over the past 3 months?")



> Entering new AgentExecutor chain...

Invoking: `get_stock_ticker_change` with `{'stockticker': 'MSFT', 'days_ago': 90}`


9.21

/home/hunglv/miniconda3/envs/llm_py310/lib/python3.10/site-packages/yfinance/utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')


The stock price of MSFT has gone up by 9.21% over the past 3 months.

> Finished chain.


'The stock price of MSFT has gone up by 9.21% over the past 3 months.'

In [40]:
open_ai_agent.run("How much has Bitcoin gone up over the past 3 months?")



> Entering new AgentExecutor chain...

Invoking: `get_stock_ticker_change` with `{'stockticker': 'BTC-USD', 'days_ago': 90}`


-1.47

/home/hunglv/miniconda3/envs/llm_py310/lib/python3.10/site-packages/yfinance/utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')


Bitcoin has gone down by 1.47% over the past 3 months.

> Finished chain.


'Bitcoin has gone down by 1.47% over the past 3 months.'